In [ ]:
!pip install --upgrade watson-developer-cloud

In [ ]:
import json
import time

## Load data from DB2 Warehouse on the Cloud to Spark Dataframe
Specify the credentials for your DB2 Warehouse on the cloud instance and read table data into Spark data frame. To do so:

- Click the Data icon (top right)
- Choose the Connections tab
- Select "Insert SparkSession DataFrame"
   - Select the correct schema
   - Choose the correct Table to upload (in our example data, it is DSX_CLOUDANT_SINGER_TWEETS)

This should copy required code into the active notebook cell for accessing your DB2 Warehouse on the Cloud instance and read the table DSX_CLOUDANT_SINGER_TWEETS into a Spark dataframe.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
# The code was removed by DSX for sharing.

## Data Exploration and Curation
Run some analysis and exploration of the data to verify it is as expected

In [ ]:
# copy data into brandTweetsDF dataframe for processing
brandTweetsDF = data_df_1

In [ ]:
# Return top 2 rows of Spark DataFrame
brandTweetsDF.limit(2).toPandas()

In [ ]:
# Print the schema of the loaded data
brandTweetsDF.printSchema()

In [ ]:
## Drop unneeded columns
brandTweetsDF = brandTweetsDF.drop('_ID','_REV')

In [ ]:
import datetime
from datetime import date
from dateutil import parser

def getDay(date):
    print('input date: ', date)
    day = parser.parse(str(date))
    day = day.date()
    return day

# Add a field for the day the tweet was created (ignoring hour/minute/second)
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType

udfGetDay = udf(getDay, DateType())

brandTweetsDF = brandTweetsDF.withColumn('DAY',udfGetDay('CREATED_AT'))

# Verify added field is as expected
brandTweetsDF.select("DAY").limit(5).toPandas()

## Extract a Random Sample of Records
Next, we will extract a randome rample of records to run NLU enrichment on. This is needed to make sure we don't exceed our limit of free NLU calls per day.

In [ ]:
## Take a sample of the data
## Limit to 1000 records as Watson NLU allows 1000 free calls per day
import random

num_records = brandTweetsDF.count()
sample_num_records = 500
fraction = float(sample_num_records)/float(num_records)

seed = random.randint(1, 100)
print('Number of records: ', num_records, ' Sample size: ', sample_num_records, ' Fraction: ', fraction, ' Seed: ', seed)
brandTweetsSampleDF = brandTweetsDF.sample(False, fraction, seed)


## Alternative Stratified Sampling approach
## Returns RDD with length of 2, first col is the key (day) and second col is the original row for the key
## Take only the actual data (column 1)
## If you'd like to use this approach, uncomment the following 4 lines

#fractionList = brandTweetsDF.rdd.map(lambda x: x['DAY']).distinct().map(lambda x: (x,fraction)).collectAsMap()
#keybyday = brandTweetsDF.rdd.keyBy(lambda x: x['DAY'])
#brandTweetsDFrdd = keybyday.sampleByKey(False,fractionList).map(lambda x: x[1])
#brandTweetsSampleDF = spark.createDataFrame(brandTweetsDFrdd,brandTweetsDF.schema)


print('Number of records to send to NLU:', brandTweetsSampleDF.count())

In [ ]:
# plot number of tweets per day
from pyspark.sql import functions as F
brandTweetsSampleDFperDay = brandTweetsSampleDF.groupBy('DAY')\
                              .agg(F.count('ID')\
                              .alias('NUM_TWEETS_PER_DAY'))
brandTweetsSampleDFperDay.show()

## Map to a Pandas dataframe and enrich with Watson Natural Language Understanding (NLU)
Note that in order to call a REST API such as NLU on the sampled records, need to map the Spark data frame to a Pandas data frame and then execute the NLU enrichment using the Pandas data frame. This effectively runs the enrichment code on the master Spark node only.

In [ ]:
# Create a Pandas frame and augment with Sentiment analysis and Keywords using Watson NLU
brandTweetsSamplePandasDF = brandTweetsSampleDF.toPandas()

## Specify NLU Credentials
Next, you need to specify the credentials for your Watson Natural Language Understanding (NLU) service. If you don't have an NLU service, you can create one by following [these instructions](https://console.bluemix.net/docs/services/natural-language-understanding/getting-started.html#getting-started-tutorial) and obtaining the service credentials. You need to specify the URL, username, and password.

In [ ]:
# Specify NLU credentials
credentials_json= {
    "nlu_url":"YOUR_NLU_URL",
    "nlu_username": "YOUR_NLU_USERNAME",
	"nlu_password": "YOUR_NLU_PASSWORD",
	"nlu_version": "2017-02-27"
}

## Watson NLU Enrichment Definition
In this cell, import the Watson Developer Cloud Python SDK, parse the NLU credentials, and define the function to enrich text with NLU.

In [ ]:
import watson_developer_cloud
#import watson_developer_cloud.natural_language_understanding.features.v1 as features
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, SentimentOptions, KeywordsOptions
from watson_developer_cloud import WatsonException

## Define credentials for NLU service
nlu_url = credentials_json['nlu_url']
nlu_username=credentials_json['nlu_username']
nlu_password=credentials_json['nlu_password']
nlu_version=credentials_json['nlu_version']
nlu = watson_developer_cloud.NaturalLanguageUnderstandingV1(version = nlu_version,
                                                            username = nlu_username,
                                                            password = nlu_password)

## Send text to NLU and extract Sentiment and Keywords
## Make sure text is utf-8 encoded
def enrichNLU(text):
    utf8text = text.encode("utf-8")
    # In python3, need to decode to string
    utf8text = utf8text.decode('utf-8')
    
    try:
        result = nlu.analyze(text = utf8text, features = Features(sentiment=SentimentOptions(),keywords=KeywordsOptions()))
        sentiment = result['sentiment']['document']['score']
        sentiment_label = result['sentiment']['document']['label']
        keywords = list(result['keywords'])  
    except WatsonException:
        result = None
        sentiment = 0.0
        sentiment_label = None
        keywords = None
    #print sentiment
    return sentiment, sentiment_label, keywords
    #return result

In [ ]:
import time
start_time = time.time()
## This calls the enrichNLU function which accesses the Watson NLU API
brandTweetsSamplePandasDF['SENTIMENT'],brandTweetsSamplePandasDF['SENTIMENT_LABEL'],\
brandTweetsSamplePandasDF['KEYWORDS'] = zip(*brandTweetsSamplePandasDF['TEXT_CLEAN'].map(enrichNLU))
#print(brandTweetsSamplePandasDF)
print('total run time for NLU enrichment using Pandas dataframe on Spark: ', time.time() - start_time)

In [ ]:
brandTweetsSamplePandasDF.count